Note: Much of this code was lifted from [the Conversation AI project](https://conversationai.github.io/).

In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import model_bias_analysis

In [2]:
## Commented out because I believe we're not using these files right now. 

# toxicity_debiasing_data = pd.read_csv("toxicity_debiasing_data.tsv", sep='\t')
# # review_id, comment, is toxic or not, split (train or test)

# toxicity_debiasing_data_random = pd.read_csv("toxicity_debiasing_data_random.tsv", sep = '\t')
# # review_id, comment, is toxic or not, split (train or test)

# wiki_debias_dev = pd.read_csv("wiki_debias_dev.csv") 
# # comment,is_toxic,logged_in,ns,rev_id,sample,split,toxicity,year

# wiki_debias_random_dev = pd.read_csv("wiki_debias_random_dev.csv")
# # comment,is_toxic,logged_in,ns,rev_id,sample,split,toxicity,year

# wiki_debias_random_test = pd.read_csv("wiki_debias_random_test.csv")
# # comment,is_toxic,logged_in,ns,rev_id,sample,split,toxicity,year

#toxicity_debiasing_data.shape#.head
#toxicity_debiasing_data_random.shape
#wiki_debias_dev.shape
#wiki_debias_random_dev.shape
#wiki_debias_random_test.shape
#sum(toxicity_debiasing_data['split'] == "test")/sum(toxicity_debiasing_data['split'] == "train")

"This data set (https://figshare.com/articles/Wikipedia_Talk_Labels_Toxicity/4563973) includes over 100k labeled discussion comments from English Wikipedia. Each comment was labeled by multiple annotators via Crowdflower on whether it is a toxic or healthy contribution. We also include some demographic data for each crowd-worker. See our wiki for documentation of the schema of each file and our research paper for documentation on the data collection and modeling methodology. For a quick demo of how to use the data for model building and analysis, check out this ipython notebook." - quote from linked page

In [3]:
comments = pd.read_csv("toxicity_annotated_comments.tsv"\
                                          , sep = "\t")

Copied from documentation: <br>
"Schema for {attack/aggression/toxicity}_annotated_comments.tsv
The comment text and metadata for comments with attack/aggression/toxicity labels generated by crowd-workers. The actual labels are in the corresponding {attack/aggression/toxicity}_annotations.tsv since each comment was labeled multiple times.

rev_id: MediaWiki revision id of the edit that added the comment to a talk page (i.e. discussion). <br>
comment: Comment text. Consists of the concatenation of content added during a revision/edit of a talk page. MediaWiki markup and HTML have been stripped out. To simplify tsv parsing, \n has been mapped to NEWLINE_TOKEN, \t has been mapped to TAB_TOKEN and " has been mapped to `. <br>
year: The year the comment was posted in. <br>
logged_in: Indicator for whether the user who made the comment was logged in. Takes on values in {0, 1}. <br>
ns: Namespace of the discussion page the comment was made in. Takes on values in {user, article}. <br>
sample: Indicates whether the comment came via random sampling of all comments, or whether it came from random sampling of the 5 comments around a block event for violating WP:npa or WP:HA. Takes on values in {random, blocked}. <br>
split: For model building in our paper we split comments into train, dev and test sets. Takes on values in {train, dev, test}."
<br>

My notes: <br> 
I don't know enough about how natural language processing works, but from the snippets that I do know, I imagine that the really really long comments probably aren't very good at being classified even. I also wonder about how bigram toxicity works and whether this is something the training data accounts for (eg "nasty woman" vs "nasty" has different sources of problems). What are they classifying on/why does this work? We can see Figure 1 in the Dixon paper for comment length and should be able to filter there. I wonder if the phrase templating of classification works for the problems that I raise of large comment length and bigrams. Is this an issue we should be dealing with?
Also, an easier couple of questions are: what does the ns and sample really mean?

In [4]:
comments.head()

,rev_id,comment,year,logged_in,ns,sample,split
0,2232.0,This:NEWLINE_TOKEN:One can make an analogy in ...,2002,True,article,random,train
1,4216.0,`NEWLINE_TOKENNEWLINE_TOKEN:Clarification for ...,2002,True,user,random,train
2,8953.0,Elected or Electoral? JHK,2002,False,article,random,test
3,26547.0,`This is such a fun entry. DevotchkaNEWLINE_...,2002,True,article,random,train
4,28959.0,Please relate the ozone hole to increases in c...,2002,True,article,random,test


In [5]:
annotations = pd.read_csv("toxicity_annotations.tsv"\
                                          , sep = "\t")

Copied from documentation:
        Schema for toxicity_annotations.tsv
    Toxicity labels from several crowd-workers for each comment in toxicity_annotated_comments.tsv. It can be joined with toxicity_annotated_comments.tsv on rev_id.

rev_id: MediaWiki revision id of the edit that added the comment to a talk page (i.e. discussion). <br>
worker_id: Anonymized crowd-worker id.<br>
toxicity_score: Categorical variable ranging from very toxic (-2), to neutral (0), to very healthy (2). <br>
toxicity: Indicator variable for whether the worker thought the comment is toxic. The annotation takes on the value 1 if the worker considered the comment toxic (i.e worker gave a toxicity_score less than 0) and value 0 if the worker considered the comment neutral or healthy (i.e worker gave a toxicity_score greater or equal to 0). Takes on values in {0, 1}.

My notes:
Things to explore is how many people rated each thing? The paper said 10, but I would like to confirm this. 

In [6]:
annotations.head()

,rev_id,worker_id,toxicity,toxicity_score
0,2232.0,723,0,0.0
1,2232.0,4000,0,0.0
2,2232.0,3989,0,1.0
3,2232.0,3341,0,0.0
4,2232.0,1574,0,1.0


In [7]:
toxicity_worker_demographics = pd.read_csv("toxicity_worker_demographics.tsv"\
                                          , sep = "\t")

My comment:
    This isn't really one we'll be using until much later, if/when we decide we're doing a perturbation using demographic data. Would first want to check to see what kind of correlations might/do exist between gender/rating and see how they rate comments about women, for example.
    

Copied from documentation:

Schema for {attack/aggression/toxicity}_worker_demographics.tsv
Demographic information about the crowdworkers. This information was obtained by an optional demographic survey administered after the labelling task. It is meant to be joined with {attack/aggression/toxicity}_annotations.tsv on worker_id. Some fields may be blank if left unanswered.

worker_id: Anonymized crowd-worker id. <br>
gender: The gender of the crowd-worker. Takes a value in {'male', 'female', and 'other'}. <br>
english_first_language: Does the crowd-worker describe English as their first language. Takes a value in {0, 1}.<br>
age_group: The age group of the crowd-worker. Takes on values in {'Under 18', '18-30', '30-45', '45-60', 'Over 60'}.<br>
education: The highest education level obtained by the crowd-worker. Takes on values in {'none', 'some', 'hs', 'bachelors', 'masters', 'doctorate', 'professional'}. Here 'none' means no schooling, some means 'some schooling', 'hs' means high school completion, and the remaining terms indicate completion of the corresponding degree type.


In [8]:
#  Stuff I tried that didn't work and I might want later

# This code is copy/pasted from https://github.com/conversationai/unintended-ml-bias-analysis/blob/master/unintended_ml_bias/Dataset_bias_analysis.ipynb
# grouped_ds = annotations.groupby('rev_id')['toxicity'].mean()
# df = comments
# df['toxic'] = annotations.groupby('rev_id')['toxicity'].mean() # > 0.5

grouped_annotations = annotations.groupby('rev_id',as_index=False)['toxicity'].mean()
joined_tox = grouped_annotations.join(comments, lsuffix='rev_id', rsuffix='rev_id', how='left', sort=True) 
joined_tox['binary_tox'] = np.where(joined_tox['toxicity']>=.5, 1, 0)
# Stuff I might want later

# # remove newline and tab tokens
# comments['comment'] = comments['comment'].apply(lambda x: x.replace("NEWLINE_TOKEN", " "))
# comments['comment'] = comments['comment'].apply(lambda x: x.replace("TAB_TOKEN", " "))
# comments['length'] = comments['comment'].str.len()

In [63]:
joined_tox.head()

,rev_idrev_id,toxicity,rev_idrev_id,comment,year,logged_in,ns,sample,split,binary_tox
0,2232.0,0.1,2232.0,This:NEWLINE_TOKEN:One can make an analogy in ...,2002,True,article,random,train,0
1,4216.0,0.0,4216.0,`NEWLINE_TOKENNEWLINE_TOKEN:Clarification for ...,2002,True,user,random,train,0
2,8953.0,0.0,8953.0,Elected or Electoral? JHK,2002,False,article,random,test,0
3,26547.0,0.0,26547.0,`This is such a fun entry. DevotchkaNEWLINE_...,2002,True,article,random,train,0
4,28959.0,0.2,28959.0,Please relate the ozone hole to increases in c...,2002,True,article,random,test,0


In [9]:
joined_tox['toxicity'].describe()


count    159686.000000
mean          0.145049
std           0.253866
min           0.000000
25%           0.000000
50%           0.000000
75%           0.200000
max           1.000000
Name: toxicity, dtype: float64

In [10]:
joined_tox['binary_tox'].value_counts()

0    141289
1     18397
Name: binary_tox, dtype: int64

In [11]:
test_comments = joined_tox.query("split == 'test' ")
train_comments = joined_tox.query("split == 'train' ")

clf = Pipeline([
    ('vect', CountVectorizer(max_features = 10000, ngram_range = (1,2))),
    ('tfidf', TfidfTransformer(norm = 'l2')),
    ('clf', LogisticRegression()),
])

clf = clf.fit(train_comments['comment'], train_comments['binary_tox'])
auc = roc_auc_score(test_comments['binary_tox'], clf.predict_proba(test_comments['comment'])[:, 1])
print('Test ROC AUC: %.3f' %auc)

Test ROC AUC: 0.951


In [12]:
test_comments["predicted"] = clf.predict(test_comments['comment'])

C:\Users\Neha\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [13]:
# An example of a comment predicted to be toxic.
test_comments[test_comments['predicted'] == 1]['comment'][1649]#['comment']#.value_counts()

"Wait a minute.  I looked at your background and I see that you are French.  I sorry.  What's it like living in a former world power that is nothing but an impotent, second-rate country with an inferiority complex???  Wait don't tell me, from your comment I already know.  It tees you off that no one and I mean no one pays any damn attention to you any more, so much so that you have to buy Saddam's oil off of the black market and invade third world countries like the Ivory Coast and then lecture Americans.  But Americans do care!!!!  We just laugh at you.  Have Frog Day!!! NEWLINE_TOKENNEWLINE_TOKEN"

In [14]:
test_comments[test_comments['predicted'] == 1].head()

,rev_idrev_id,toxicity,rev_idrev_id,comment,year,logged_in,ns,sample,split,binary_tox,predicted
829,5815399.0,0.7,5815399.0,NEWLINE_TOKENNEWLINE_TOKENIt was anonymous on ...,2004,True,article,random,test,1,1
1227,8357736.0,0.3,8357736.0,NEWLINE_TOKENNEWLINE_TOKENYou are a child.,2004,True,user,blocked,test,0,1
1228,8359431.0,0.5,8359431.0,`NEWLINE_TOKENNEWLINE_TOKEN::You are not worth...,2004,True,user,blocked,test,1,1
1348,8845700.0,1.0,8845700.0,NEWLINE_TOKENNEWLINE_TOKENi am using the sandb...,2004,False,user,blocked,test,1,1
1521,9664203.0,0.8,9664203.0,NEWLINE_TOKENNEWLINE_TOKEN== whoa ==NEWLINE_TO...,2005,False,user,blocked,test,1,1


In [15]:
print(clf.predict(['Thanks for you contribution, you did a great job!']))
# Model labels this comment as non-toxic.

[0]


In [16]:
print(clf.predict(['People as stupid as you should not edit Wikipedia!']))
# Model labels this comment as toxic.

[1]


In [17]:
print(clf.predict(['I am a proud gay']))
# Model also labels this comment as toxic. Uh-oh! We don't think this comment should be toxic!

[1]


In [18]:
TEST_TERMS = ['tall', 'wikipedia', 'teacher', 'music', 'box',
              'lesbian', 'gay', 'bisexual', 'transgender', 'queer',
              'homosexual', 'heterosexual', 'straight',
              'muslim', 'jewish', 'jew', 'christian',
              'feminist', 'black', 'white']

In [19]:
madlibs_terms = model_bias_analysis.read_identity_terms('adjectives_people.txt')

In [20]:
model_bias_analysis.add_subgroup_columns_from_text(train_comments, 'comment', madlibs_terms)

C:\Users\Neha\Documents\GitHub\cs590_privacy_fairness\model_bias_analysis.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[term] = df[text_column].apply(lambda x: bool(re.search(r'\b{}\b'.format(term), x, flags=re.IGNORECASE)))


In [21]:
model_bias_analysis.add_subgroup_columns_from_text(test_comments, 'comment', madlibs_terms)

C:\Users\Neha\Documents\GitHub\cs590_privacy_fairness\model_bias_analysis.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[term] = df[text_column].apply(lambda x: bool(re.search(r'\b{}\b'.format(term), x, flags=re.IGNORECASE)))


In [22]:
train_comments.head()

,rev_idrev_id,toxicity,rev_idrev_id,comment,year,logged_in,ns,sample,split,binary_tox,...,older,young,younger,teenage,millenial,middle aged,elderly,blind,deaf,paralyzed
0,2232.0,0.1,2232.0,This:NEWLINE_TOKEN:One can make an analogy in ...,2002,True,article,random,train,0,...,False,False,False,False,False,False,False,False,False,False
1,4216.0,0.0,4216.0,`NEWLINE_TOKENNEWLINE_TOKEN:Clarification for ...,2002,True,user,random,train,0,...,False,False,False,False,False,False,False,False,False,False
3,26547.0,0.0,26547.0,`This is such a fun entry. DevotchkaNEWLINE_...,2002,True,article,random,train,0,...,False,False,False,False,False,False,False,False,False,False
6,37330.0,0.3,37330.0,`NEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENI fixe...,2002,True,article,random,train,0,...,False,False,False,False,False,False,False,False,False,False
7,37346.0,0.1,37346.0,`If they are ``indisputable`` then why does th...,2002,True,article,random,train,0,...,False,False,False,False,False,False,False,False,False,False


In [23]:
test_comments.head()

,rev_idrev_id,toxicity,rev_idrev_id,comment,year,logged_in,ns,sample,split,binary_tox,...,older,young,younger,teenage,millenial,middle aged,elderly,blind,deaf,paralyzed
2,8953.0,0.0,8953.0,Elected or Electoral? JHK,2002,False,article,random,test,0,...,False,False,False,False,False,False,False,False,False,False
4,28959.0,0.2,28959.0,Please relate the ozone hole to increases in c...,2002,True,article,random,test,0,...,False,False,False,False,False,False,False,False,False,False
19,138074.0,0.0,138074.0,`NEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLIN...,2002,True,article,random,test,0,...,False,False,False,False,False,False,False,False,False,False
33,200664.0,0.0,200664.0,NEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKEN NEWLIN...,2002,True,user,random,test,0,...,False,False,False,False,False,False,False,False,False,False
37,213105.0,0.0,213105.0,`NEWLINE_TOKENNEWLINE_TOKEN: I should do that ...,2002,True,user,random,test,0,...,False,False,False,False,False,False,False,False,False,False


In [24]:
#  numerator train
train_comments[train_comments['lesbian'] == True].shape[0]

38

In [25]:
# training data comments containing 'lesbian' AND that are toxic
# denominator train
train_comments[(train_comments['lesbian'] == True) & (train_comments['binary_tox'] == 1)].shape[0]

15

In [77]:
# We now calculate the DI for each pair for the training data.

proportion_toxic_by_term = np.zeros(len(madlibs_terms))

# we temporarily put -1 in as a placeholder for stuff that has no exacmples of terms with that term.
# non-binary is one
for i in range(len(madlibs_terms)):
    try:
        proportion_toxic_by_term[i] = (train_comments[(train_comments[madlibs_terms[i]] == True) & (train_comments['binary_tox'] == 1)].shape[0])/  \
        (train_comments[train_comments[madlibs_terms[i]] == True].shape[0])
    except ZeroDivisionError:
        proportion_toxic_by_term[i] = -1 
        
print(proportion_toxic_by_term)
# Possible thing to investigate -- why are the high values so high? For example, number 48.

[ 0.39473684  0.62544803  0.07692308  0.26315789  0.0952381   0.74509804
  0.07017544  1.          0.46774194  0.13702624  0.15789474  0.17073171
  0.15577889 -1.          0.08474576  0.16326531  0.18042813  0.15853659
  0.09090909  0.10416667  0.11111111  0.2        -1.          0.2739726
  0.06944444  0.09857978  0.12        0.11444142  0.09090909  0.12058824
  0.07216495  0.09190372  0.13057325  0.11860465  0.02173913  0.08906883
  0.03846154  0.11111111 -1.          0.12916667  0.0472103   0.09433962
  0.03529412  0.125       0.         -1.          0.          0.19496855
  0.08       -1.        ]


In [78]:
# This is number 48's numerator, which is deaf
train_comments[(train_comments[madlibs_terms[48]] == True) & (train_comments['binary_tox'] == 1)]

,rev_idrev_id,toxicity,rev_idrev_id,comment,year,logged_in,ns,sample,split,binary_tox,...,older,young,younger,teenage,millenial,middle aged,elderly,blind,deaf,paralyzed
6969,29352085.0,0.7,29352085.0,"This user is deaf discriminator, facist, and r...",2005,True,user,blocked,train,1,...,False,False,False,False,False,False,False,False,True,False
50271,165982480.0,0.6,165982480.0,NEWLINE_TOKENNEWLINE_TOKEN:Screw the fact that...,2007,True,article,blocked,train,1,...,False,False,False,False,False,False,False,True,True,False


In [79]:
# The proportion is definitely 12.5
proportion_toxic_by_term[48]

0.08

In [75]:
train_comments[(train_comments[madlibs_terms[48]] == True)]

,rev_idrev_id,toxicity,rev_idrev_id,comment,year,logged_in,ns,sample,split,binary_tox,...,older,young,younger,teenage,millenial,middle aged,elderly,blind,deaf,paralyzed
4801,21778549.0,0.0,21778549.0,NEWLINE_TOKENNEWLINE_TOKEN==Vandalism (in an o...,2005,True,user,random,train,0,...,False,False,False,False,False,False,False,False,True,False
5196,22870467.0,0.0,22870467.0,`Danny you said above about my complaint of PO...,2005,True,article,blocked,train,0,...,False,False,False,False,False,False,False,False,True,False
6891,29132068.0,0.1,29132068.0,NEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENChapsNE...,2005,True,user,random,train,0,...,False,False,False,False,False,False,False,False,True,False
6969,29352085.0,0.7,29352085.0,"This user is deaf discriminator, facist, and r...",2005,True,user,blocked,train,1,...,False,False,False,False,False,False,False,False,True,False
11844,42151395.0,0.1,42151395.0,REDIRECT Wikipedia talk:WikiProject Deaf,2006,True,user,random,train,0,...,False,False,False,False,False,False,False,False,True,False
14188,48299103.0,0.1,48299103.0,`NEWLINE_TOKENNEWLINE_TOKEN==Serbian casualtie...,2006,True,article,blocked,train,0,...,False,False,False,False,False,False,False,True,True,False
21988,67726771.0,0.2,67726771.0,But ur not deaf! I also have the full tamar Br...,2006,True,user,blocked,train,0,...,False,False,False,False,False,False,False,False,True,False
24531,74350281.0,0.4,74350281.0,`NEWLINE_TOKENNEWLINE_TOKEN== No apology ==NEW...,2006,True,user,blocked,train,0,...,False,False,False,False,False,False,False,True,True,False
24699,74755861.0,0.3,74755861.0,"Obviously, most admins don't have the time...",2006,False,user,blocked,train,0,...,False,False,False,False,False,False,False,False,True,False
25638,77450729.0,0.3,77450729.0,`JokesNEWLINE_TOKENHow does Helen Keller drive...,2006,False,article,blocked,train,0,...,False,False,False,False,False,False,False,False,True,False


In [38]:

# numerator test
test_comments[test_comments['lesbian'] == True].shape[0]

18

In [ ]:
# test data comments containing 'lesbian' AND that are toxic
# denominator test
test_comments[(test_comments['lesbian'] == True) & (test_comments['predicted'] == 1)].shape[0]